In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas
import plotly.express as px
import os
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
import imageio
import mapclassify
import matplotlib
import matplotlib.image as mpimg
import ast
import json
import numpy as np
import shapely.wkt
import math

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib
pd.set_option('display.max_columns', None)
from six.moves import reduce
import scipy
from sklearn.metrics import r2_score
#scipy.stats.pearsonr
import scipy.stats as stats
plt.rcParams.update({'font.family': 'Arial'})
import matplotlib.colors as clr
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import TwoSlopeNorm
from sklearn.cluster import *
from sklearn.decomposition import PCA
from functools import reduce
import scipy.stats

import math
import sklearn
import sklearn.feature_extraction.text
from sklearn import cluster
from sklearn import metrics
import sklearn.manifold #For T-SNE
import sklearn.decomposition #For PCA
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.decomposition
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.neural_network
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.metrics import roc_auc_score
from shapely import wkt
#from mycolorpy import colorlist as mcp
import matplotlib.ticker as mtick
from  matplotlib.colors import LinearSegmentedColormap
from functools import reduce
import matplotlib.ticker as plticker
from functools import reduce

In [2]:
variables = ['H_Area_of_store', 
       'R_Percentage_of_Visits_by_brand', 'J_POI_count_where_store_is',
       'K_POI_diversity_where_store_is', 'L_Demographic_similarity',
       'G_Distance_between_cbg_and_store']

In [3]:
vars_definition = {'H_Area_of_store': 'Store area', 
       'R_Percentage_of_Visits_by_brand': 'Chain loyalty', 'J_POI_count_where_store_is': 'POI count',
       'K_POI_diversity_where_store_is': 'POI diversity', 'L_Demographic_similarity': 'Demographic similarity',
       'G_Distance_between_cbg_and_store':'CBG-Store Distance'}

In [4]:
ses = ['med_ageE','bachelor_degree_25E', 'med_hh_incomeE', 'whiteE', 'blackE', 'asianE','hispanicE']

In [5]:
census = pd.read_csv('data/1_data_for_extracting_information/NY_cbg_census.csv').rename(
    columns = {'GEOID':'cbg'})

param_2018 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2018/PSO_2018_6params_NYC_norm_28_PSO_15.csv')

param_2019 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2019/PSO_2019_6params_NYC_norm_28_PSO_15.csv')

param_2020 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2020/PSO_2020_6params_NYC_norm_28_PSO_15.csv')

param_2021 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2021/PSO_2021_6params_NYC_norm_28_PSO_15.csv')

dfs_hist = {'param_2018': param_2018,'param_2019': param_2019,'param_2020': param_2020, 'param_2021': param_2021}

for k,v in dfs_hist.items():
    v['year']=[int(k.split('_')[-1])]*len(v)
    
params_all = pd.concat([param_2018, param_2019, param_2020, param_2021]).reset_index().drop(columns = 'index')

params_all = params_all[params_all['H_Area_of_store'] != 'no visitors']
params_all[variables] = params_all[variables].astype(float)

In [6]:
def get_pro(part):
    total_list = census.populationE.tolist()
    part_list = census[part].tolist()
    ans = []
    for i,num in enumerate(total_list):
        if num == 0:
            ans.append(0)
        else:
            ans.append(part_list[i]/num)
    return ans

In [7]:
change = ['bachelor_degree_25E', 'whiteE', 'blackE', 'asianE', 'hispanicE']
for i in change:
    census[i] = get_pro(i)

In [8]:
params_all = params_all.merge(census[['cbg']+ses], how = 'inner').dropna()

In [9]:
params_all['year'].value_counts()

2018    5185
2019    5185
2020    5185
2021    5185
Name: year, dtype: int64

In [10]:
def get_single_top_stats_that_year(ses_var, prop, year_now):
    
    d_temp_this_year = params_all[params_all['year'] == year_now]
    top_d_temp_this_year = d_temp_this_year[d_temp_this_year['cbg'].isin(d_temp_this_year.nlargest(int(len(
        d_temp_this_year)*prop), ses_var)['cbg'].tolist())]
    return top_d_temp_this_year[variables].mean().reset_index().rename(columns = {
        'index': 'variable', 0: '5%_' + ses_var})

get_single_top_stats_that_year(ses[0], 0.05, 2019)

,variable,5%_med_ageE
0,H_Area_of_store,3.643985
1,R_Percentage_of_Visits_by_brand,6.488596
2,J_POI_count_where_store_is,3.925475
3,K_POI_diversity_where_store_is,7.472741
4,L_Demographic_similarity,8.339997
5,G_Distance_between_cbg_and_store,10.452566


In [11]:
def get_stats_that_year(prop, year_now):
    general_stats = params_all[params_all['year'] == year_now].mean().reset_index().rename(
        columns = {'index': 'variable', 0: 'general'})
    data_here = [general_stats]
    for s in ses:
        data_here.append(get_single_top_stats_that_year(s, prop, year_now))
        
    return reduce(lambda x, y: pd.merge(x, y, on = 'variable'), data_here)
stats_2019 = get_stats_that_year(0.05, 2019)
stats_2019

,variable,general,5%_med_ageE,5%_bachelor_degree_25E,5%_med_hh_incomeE,5%_whiteE,5%_blackE,5%_asianE,5%_hispanicE
0,H_Area_of_store,4.179320,3.643985,2.673862,3.141165,3.436067,5.119034,3.914596,5.164255
1,R_Percentage_of_Visits_by_brand,6.435345,6.488596,8.247287,8.127159,6.678476,7.058363,5.027343,5.838563
2,J_POI_count_where_store_is,3.779453,3.925475,4.077364,4.140112,4.199676,3.711623,4.259771,3.898188
3,K_POI_diversity_where_store_is,6.724573,7.472741,7.574158,7.894333,7.523606,5.480276,7.300896,6.577857
4,L_Demographic_similarity,7.882446,8.339997,8.515744,8.260572,8.087285,7.572610,8.142435,7.878695
5,G_Distance_between_cbg_and_store,10.651235,10.452566,10.650228,10.411821,10.230154,10.782914,10.372933,11.133773


In [12]:
stats_2019.to_csv(
    'data/4_analysis/parameter_values_ses_cluster_comparison_2019.csv', 
    index = False)

In [13]:
for indx, cont in stats_2019.iterrows():
    values = [cont['general'], cont['5%_med_ageE'], cont['5%_bachelor_degree_25E'], 
              cont['5%_med_hh_incomeE'], cont['5%_whiteE'], cont['5%_blackE'], cont['5%_asianE'],
             cont['5%_hispanicE']]
    values = [str(round(i, 3)) for i in values]
    print(' & '.join(values))
    print('\n')

4.179 & 3.644 & 2.674 & 3.141 & 3.436 & 5.119 & 3.915 & 5.164


6.435 & 6.489 & 8.247 & 8.127 & 6.678 & 7.058 & 5.027 & 5.839


3.779 & 3.925 & 4.077 & 4.14 & 4.2 & 3.712 & 4.26 & 3.898


6.725 & 7.473 & 7.574 & 7.894 & 7.524 & 5.48 & 7.301 & 6.578


7.882 & 8.34 & 8.516 & 8.261 & 8.087 & 7.573 & 8.142 & 7.879


10.651 & 10.453 & 10.65 & 10.412 & 10.23 & 10.783 & 10.373 & 11.134


